In [1]:
# Install required packages with specific compatible versions
!pip install numpy==1.26.4 pandas==2.2.2 scipy==1.13.1 statsmodels==0.14.2 matplotlib==3.7.1 seaborn==0.13.2 scikit-learn==1.5.1 qiskit qiskit-aer

import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import os
import time
import csv
from datetime import datetime
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import warnings

# Suppress RuntimeWarnings
warnings.filterwarnings('ignore', message='invalid value encountered in multiply')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: matplotlib
   

In [2]:
def generate_prng_data(num_samples=10000, seed=42):
    """Generate PRNG data with 32-bit integers and doubles [0, 1]."""
    np.random.seed(seed)
    integers = np.random.randint(0, 2**32, num_samples, dtype=np.uint32)
    doubles = integers / (2**32 - 1)

    start_time = time.time()
    data = []
    for i in range(num_samples):
        timestamp = datetime.now().timestamp()
        data.append({
            'index': i,
            'integer': int(integers[i]),
            'double': float(doubles[i]),
            'source': 'PRNG',
            'timestamp': timestamp
        })

    output_file = 'prng_data.csv'
    try:
        with open(output_file, 'w', newline='') as csvfile:
            fieldnames = ['index', 'integer', 'double', 'source', 'timestamp']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)
        print(f"PRNG data saved to {output_file} with {num_samples} samples")
    except IOError as e:
        print(f"Error writing PRNG data: {e}")
        return

    exec_time = time.time() - start_time
    time_file = 'prng_time.txt'
    try:
        with open(time_file, 'w') as f:
            f.write(str(exec_time))
        print(f"PRNG execution time saved to {time_file}: {exec_time:.2f} seconds")
    except IOError as e:
        print(f"Error writing PRNG time: {e}")

    return exec_time

# Generate PRNG data
print("Generating PRNG data (10,000 samples)...")
prng_time = generate_prng_data()

Generating PRNG data (10,000 samples)...
PRNG data saved to prng_data.csv with 10000 samples
PRNG execution time saved to prng_time.txt: 0.12 seconds


In [3]:
def create_quantum_circuit(num_qubits):
    """Create a quantum circuit with Hadamard gates on all qubits."""
    qc = QuantumCircuit(num_qubits, num_qubits)
    for i in range(num_qubits):
        qc.h(i)  # Apply Hadamard gate
    qc.measure_all()  # Measure all qubits
    return qc

def bits_to_integers(bitstring, bits_per_int=32):
    """Convert a bitstring into 32-bit integers."""
    integers = []
    for i in range(0, len(bitstring) - bits_per_int + 1, bits_per_int):
        chunk = bitstring[i:i + bits_per_int]
        if len(chunk) == bits_per_int:
            integers.append(int(chunk, 2))
    return integers

def integers_to_doubles(integers):
    """Convert 32-bit integers to doubles in [0, 1]."""
    max_int = 2**32 - 1
    return [x / max_int for x in integers]

def run_quantum_simulator(num_samples=10000, num_qubits=6, shots_per_run=1000):
    """Generate quantum random numbers using Qiskit Aer simulator with optimized parameters."""
    start_time = time.time()
    max_execution_time = 3600  # 1-hour cap

    total_bits_needed = num_samples * 32
    bits_per_run = num_qubits * shots_per_run
    runs = int(np.ceil(total_bits_needed / bits_per_run))
    print(f"Using {num_qubits} qubits, {runs} runs, {shots_per_run} shots per run")

    simulator = AerSimulator()
    all_bits = ""

    for _ in range(runs):
        if time.time() - start_time > max_execution_time:
            print("Execution time limit of 1 hour reached.")
            break

        qc = create_quantum_circuit(num_qubits)
        job = simulator.run(qc, shots=shots_per_run)
        result = job.result()
        counts = result.get_counts()

        bitstring = max(counts, key=counts.get).replace(" ", "")
        all_bits += bitstring * shots_per_run

    bitstring = all_bits[:total_bits_needed]
    integers = bits_to_integers(bitstring)
    doubles = integers_to_doubles(integers)

    if len(integers) < num_samples:
        print(f"Warning: Only generated {len(integers)} numbers instead of {num_samples}")
        additional = num_samples - len(integers)
        pad_ints = np.random.randint(0, 2**32, additional, dtype=np.uint32)
        pad_doubles = pad_ints / (2**32 - 1)
        integers = np.concatenate([integers, pad_ints])
        doubles = np.concatenate([doubles, pad_doubles])
    elif len(integers) > num_samples:
        integers = integers[:num_samples]
        doubles = doubles[:num_samples]

    data = []
    for i in range(num_samples):
        timestamp = datetime.now().timestamp()
        data.append({
            'index': i,
            'integer': int(integers[i]),
            'double': float(doubles[i]),
            'source': 'Quantum',
            'timestamp': timestamp
        })

    output_file = 'quantum_data.csv'
    try:
        with open(output_file, 'w', newline='') as csvfile:
            fieldnames = ['index', 'integer', 'double', 'source', 'timestamp']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)
        print(f"Quantum data saved to {output_file} with {num_samples} samples")
    except IOError as e:
        print(f"Error writing Quantum data: {e}")
        return

    exec_time = time.time() - start_time
    time_file = 'quantum_time.txt'
    try:
        with open(time_file, 'w') as f:
            f.write(str(exec_time))
        print(f"Quantum execution time saved to {time_file}: {exec_time:.2f} seconds")
    except IOError as e:
        print(f"Error writing Quantum time: {e}")

    return exec_time

# Generate Quantum data with optimized parameters
print("\nGenerating Quantum simulated data (10,000 samples)...")
quantum_time = run_quantum_simulator()


Generating Quantum simulated data (10,000 samples)...
Using 6 qubits, 54 runs, 1000 shots per run
Quantum data saved to quantum_data.csv with 10000 samples
Quantum execution time saved to quantum_time.txt: 1.07 seconds


In [4]:
# Bootstrapping and analysis functions
def bootstrap_metric(data, metric_func, n_bootstraps=1000):
    """Perform bootstrapping on data to compute mean and 95% confidence intervals."""
    boot_results = []
    for i in range(n_bootstraps):
        sample = np.random.choice(data, size=len(data), replace=True)
        try:
            result = metric_func(sample)
            if not np.isnan(result):
                boot_results.append(result)
            else:
                print(f"Warning: NaN result in bootstrap iteration {i} for {metric_func.__name__}")
        except Exception as e:
            print(f"Warning: Chi-square failure in bootstrap iteration {i}: {e}")
            continue
    if not boot_results:
        return np.nan, (np.nan, np.nan)
    if len(np.unique(boot_results)) == 1:
        print(f"Warning: Zero variance in bootstrapped results for {metric_func.__name__}")
        return boot_results[0], (np.nan, np.nan)
    mean = np.mean(boot_results)
    ci = stats.t.interval(0.95, df=len(boot_results)-1, loc=mean, scale=stats.sem(boot_results)) if len(boot_results) > 1 else (np.nan, np.nan)
    return mean, ci

def quality_score(entropy, chi2_p, autocorr):
    """Compute a quality score based on raw randomness metrics."""
    return 0.4 * entropy + 0.4 * chi2_p - 0.2 * np.abs(autocorr)

def calculate_throughput(ints, exec_time):
    """Calculate throughput in bits per second."""
    return len(ints) * 32 / exec_time if exec_time else None

def bootstrap_metrics(ints, doubles, bin_edges, expected):
    """Compute bootstrapped metrics for entropy, chi-square p-value, and autocorrelation."""
    # Entropy for integers
    entropy_int_boot, entropy_int_ci = bootstrap_metric(
        ints,
        lambda x: -np.sum(
            (p := np.bincount(np.frombuffer(x.astype(np.uint32).tobytes(), dtype=np.uint8), minlength=256) / len(x.astype(np.uint32).tobytes())) *
            np.log2(np.where(p > 0, p, 1))
        )
    )

    # Chi-square p-value for integers with stability check
    chi2_p_boot, chi2_p_ci = bootstrap_metric(
        ints,
        lambda x: stats.chisquare(
            np.histogram(x, bins=bin_edges)[0],
            expected,
            ddof=0
        )[1] if len(np.unique(np.histogram(x, bins=bin_edges)[0])) > 1 else np.nan
    )

    # Autocorrelation (lag-1) for doubles
    autocorr_boot, autocorr_ci = bootstrap_metric(
        doubles,
        lambda x: acf(x, nlags=1, fft=True)[1]
    )

    return (entropy_int_boot, entropy_int_ci), (chi2_p_boot, chi2_p_ci), (autocorr_boot, autocorr_ci)

def compute_quality_scores(entropy_int, p_value, autocorr, entropy_int_boot, chi2_p_boot, autocorr_boot):
    """Compute original and bootstrapped quality scores."""
    quality = quality_score(entropy_int, p_value, autocorr)
    quality_boot = quality_score(entropy_int_boot, chi2_p_boot, autocorr_boot)
    return quality, quality_boot

def collect_results(source, entropy_int, entropy_double, p_value, autocorr, throughput, exec_time,
                    entropy_int_boot, entropy_int_ci, chi2_p_boot, chi2_p_ci, autocorr_boot, autocorr_ci,
                    quality, quality_boot, results_list):
    """Append analysis results to the results list."""
    results_list.append([source, entropy_int, entropy_double, p_value, autocorr, throughput, exec_time, quality,
                         entropy_int_boot, entropy_int_ci, chi2_p_boot, chi2_p_ci, autocorr_boot, autocorr_ci, quality_boot])

# Load and analyze data
files = ['prng_data.csv', 'quantum_data.csv']
dataframes = []
for f in files:
    if os.path.exists(f):
        df = pd.read_csv(f)
        dataframes.append(df)
        print(f"Loaded {f} with {len(df)} samples")
    else:
        print(f"Warning: {f} not found, skipping.")
if not dataframes:
    print("Error: No data files found.")
    exit(1)
data = pd.concat(dataframes, ignore_index=True)

# Read execution times
exec_times = {}
for method, fname in [('PRNG', 'prng_time.txt'), ('Quantum', 'quantum_time.txt')]:
    if os.path.exists(fname):
        with open(fname, 'r') as f:
            exec_times[method] = float(f.read().strip())
    else:
        exec_times[method] = {'PRNG': 0.27, 'Quantum': 1.94}.get(method, None)
        print(f"Warning: {fname} not found, using fallback execution time: {exec_times[method]} seconds")

# Perform analysis
results = []
for source in ['PRNG', 'Quantum']:
    subset = data[data['source'] == source]
    ints = subset['integer'].values
    doubles = subset['double'].values

    # Original metrics
    int_bytes = ints.astype(np.uint32).tobytes()
    byte_counts = np.bincount(np.frombuffer(int_bytes, dtype=np.uint8), minlength=256)
    p = byte_counts / len(int_bytes)
    entropy_int = -np.sum(p[p > 0] * np.log2(p[p > 0]))

    double_bytes = doubles.tobytes()
    byte_counts = np.bincount(np.frombuffer(double_bytes, dtype=np.uint8), minlength=256)
    p = byte_counts / len(double_bytes)
    entropy_double = -np.sum(p[p > 0] * np.log2(p[p > 0]))

    bin_edges = np.linspace(0, 2**32, 21)  # Increased to 20 bins for better resolution
    observed, _ = np.histogram(ints, bins=bin_edges)
    expected = np.full(20, len(ints) / 20)
    chi2, p_value = stats.chisquare(observed, expected)

    autocorr = acf(doubles, nlags=1, fft=True)[1]

    exec_time = exec_times.get(source, None)
    throughput = calculate_throughput(ints, exec_time)

    (entropy_int_boot, entropy_int_ci), (chi2_p_boot, chi2_p_ci), (autocorr_boot, autocorr_ci) = bootstrap_metrics(ints, doubles, bin_edges, expected)
    quality, quality_boot = compute_quality_scores(entropy_int, p_value, autocorr, entropy_int_boot, chi2_p_boot, autocorr_boot)
    collect_results(source, entropy_int, entropy_double, p_value, autocorr, throughput, exec_time,
                    entropy_int_boot, entropy_int_ci, chi2_p_boot, chi2_p_ci, autocorr_boot, autocorr_ci,
                    quality, quality_boot, results)

# Normalize quality scores
scaler = MinMaxScaler()
quality_values = [r[7] for r in results] + [r[14] for r in results]
normalized_qualities = scaler.fit_transform(np.array(quality_values).reshape(-1, 1)).flatten()
for i, r in enumerate(results):
    r[7] = normalized_qualities[i]
    r[14] = normalized_qualities[i + len(results)]

# Save and display results
columns = ['method', 'entropy_int', 'entropy_double', 'chi_square_p', 'autocorr_lag1', 'throughput', 'exec_time', 'quality',
           'entropy_int_boot', 'entropy_int_ci', 'chi_square_p_boot', 'chi_square_p_ci', 'autocorr_lag1_boot', 'autocorr_lag1_ci', 'quality_boot']
pd.DataFrame(results, columns=columns).to_csv('prng_quantum_analysis_summary.csv', index=False)

# Plots
plt.figure(figsize=(12, 6))
for i, source in enumerate(['PRNG', 'Quantum']):
    plt.hist(data[data['source'] == source]['integer'], bins=100, alpha=0.5, density=False, label=source, color=plt.cm.Set1(i))
plt.legend()
plt.title('Histogram of Integers: PRNG vs. Quantum')
plt.xlabel('Integer Value (0–4,294,967,295)')
plt.ylabel('Frequency')
plt.yscale('log')
plt.savefig('prng_quantum_integers_hist_enhanced.png')
plt.close()

plt.figure(figsize=(12, 6))
for i, source in enumerate(['PRNG', 'Quantum']):
    doubles = data[data['source'] == source]['double'].values
    autocorr = acf(doubles, nlags=20, fft=True)
    n = len(doubles)
    ci = 1.96 / np.sqrt(n)
    plt.plot(range(21), autocorr, label=source, color=plt.cm.Set1(i))
    plt.fill_between(range(21), autocorr - ci, autocorr + ci, alpha=0.2, color=plt.cm.Set1(i))
plt.legend()
plt.title('Autocorrelation of Doubles: PRNG vs. Quantum (Lags 0–20)')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.savefig('prng_quantum_autocorr_plot_enhanced.png')
plt.close()

# Statistical tests
from scipy.stats import kstest, uniform
print("\nKolmogorov-Smirnov Test for Uniformity (PRNG vs. Quantum):")
for source in ['PRNG', 'Quantum']:
    subset = data[data['source'] == source]['integer'].values / 2**32
    stat, p = kstest(subset, 'uniform')
    print(f"{source}: Statistic = {stat:.4f}, p-value = {p:.4f}")

print("\nAnalysis Summary (PRNG vs. Quantum):")
for r in results:
    print(f"Method: {r[0]}")
    print(f"  Entropy (int): {r[1]:.4f} bits/byte")
    print(f"  Entropy (double): {r[2]:.4f} bits/byte")
    print(f"  Chi-square p-value: {r[3]:.4f}")
    print(f"  Autocorr lag-1: {r[4]:.4f}")
    print(f"  Throughput: {r[5]:.2f} bits/second")
    print(f"  Exec time: {r[6]:.2f} seconds")
    print(f"  Quality score: {r[7]:.4f}")
    print(f"  Bootstrapped Entropy (int): {r[8]:.4f} ({r[9][0]:.4f} - {r[9][1]:.4f})")
    print(f"  Bootstrapped Chi-square p-value: {r[10]:.4f} ({r[11][0]:.4f} - {r[11][1]:.4f})")
    print(f"  Bootstrapped Autocorr lag-1: {r[12]:.4f} ({r[13][0]:.4f} - {r[13][1]:.4f})")
    print(f"  Bootstrapped Quality score: {r[14]:.4f}")

Loaded prng_data.csv with 10000 samples
Loaded quantum_data.csv with 10000 samples

Kolmogorov-Smirnov Test for Uniformity (PRNG vs. Quantum):
PRNG: Statistic = 0.0064, p-value = 0.8080
Quantum: Statistic = 0.3551, p-value = 0.0000

Analysis Summary (PRNG vs. Quantum):
Method: PRNG
  Entropy (int): 7.9956 bits/byte
  Entropy (double): 7.4575 bits/byte
  Chi-square p-value: 0.8072
  Autocorr lag-1: -0.0009
  Throughput: 2582197.66 bits/second
  Exec time: 0.12 seconds
  Quality score: 1.0000
  Bootstrapped Entropy (int): 7.9910 (7.9910 - 7.9911)
  Bootstrapped Chi-square p-value: 0.1058 (0.0954 - 0.1162)
  Bootstrapped Autocorr lag-1: -0.0003 (-0.0009 - 0.0003)
  Bootstrapped Quality score: 0.8360
Method: Quantum
  Entropy (int): 4.6380 bits/byte
  Entropy (double): 5.3063 bits/byte
  Chi-square p-value: 0.0000
  Autocorr lag-1: -0.2784
  Throughput: 300155.53 bits/second
  Exec time: 1.07 seconds
  Quality score: 0.0000
  Bootstrapped Entropy (int): 4.6372 (4.6369 - 4.6376)
  Bootstrap

In [ ]:
# Install required packages in Colab with compatible versions
!pip install qiskit==1.4.0 qiskit-ibm-runtime==0.36.1 numpy==1.26.4

import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Batch, SamplerV2 as Sampler
import time
import csv
import os

# Authenticate with IBM Quantum API token
api_token = 'YOUR_API_TOKEN'  # Replace with your actual IBM Quantum API token from quantum-computing.ibm.com
try:
    service = QiskitRuntimeService(channel="ibm_quantum", token=api_token)
    print("IBM Runtime Service loaded successfully.")
except Exception as e:
    print(f"Authentication failed: {e}")
    print("Ensure your API token is correct and pasted above.")
    raise

def create_quantum_circuit(num_qubits):
    """Create a quantum circuit with Hadamard gates on all qubits."""
    qc = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        qc.h(i)  # Apply Hadamard gate
    qc.measure_all()  # Measure all qubits
    return qc

def bits_to_integers(bitstring, bits_per_int=32):
    """Convert a bitstring into 32-bit integers."""
    integers = []
    for i in range(0, len(bitstring) - bits_per_int + 1, bits_per_int):
        chunk = bitstring[i:i + bits_per_int]
        if len(chunk) == bits_per_int:
            integers.append(int(chunk, 2))
    return integers

def integers_to_doubles(integers):
    """Convert 32-bit integers to doubles in [0, 1]."""
    max_int = 2**32 - 1
    return [x / max_int for x in integers]

def run_quantum_hardware_qrng(num_samples=500, num_qubits=1, shots_per_job=250, num_circuits=64):
    """Generate 500 32-bit quantum random numbers using IBM Quantum hardware in batch mode."""
    start_time = time.time()
    max_execution_time = 600  # 600 seconds cap per job
    total_bits_needed = num_samples * 32  # 16,000 bits for 500 samples

    print(f"Batch run: {num_circuits} circuits, {num_qubits} qubit each, {shots_per_job} shots per circuit")
    print(f"Generating {total_bits_needed} bits for {num_samples} 32-bit integers.")
    print("Queue times may extend beyond 600 seconds.")

    # Select a backend
    try:
        backend = service.least_busy(operational=True, simulator=False)
        print(f"Selected backend: {backend.name}")
    except Exception as e:
        print(f"Error selecting backend: {e}")
        print("List available backends with: [b.name for b in service.backends()]")
        return

    # Generate and transpile circuits
    circuits = [create_quantum_circuit(num_qubits) for _ in range(num_circuits)]
    transpiled_circuits = [transpile(circ, backend=backend, optimization_level=3) for circ in circuits]
    print("Circuits transpiled successfully.")

    all_bits = ""
    jobs = []

    # Run jobs in batch mode
    try:
        with Batch(backend=backend, max_time="10m"):  # 10-minute max TTL for the batch
            sampler = Sampler()
            for i, transpiled_qc in enumerate(transpiled_circuits):
                job = sampler.run([transpiled_qc], shots=shots_per_job)
                jobs.append(job)
                print(f"Job {i+1} submitted: {job.job_id()}")

        # Wait for all jobs to complete and collect results
        for i, job in enumerate(jobs):
            result = job.result(timeout=max_execution_time)
            counts = result[0].data.meas.get_counts()
            print(f"Job {i+1} completed with counts: {counts}")
            for bitstring in counts:
                all_bits += bitstring * counts[bitstring]

    except Exception as e:
        print(f"Batch execution failed: {e}")
        if "timeout" in str(e).lower():
            print("Execution exceeded 600 seconds; check job status on IBM Quantum dashboard.")
        elif "quota" in str(e).lower():
            print("Quota exceeded; wait for daily refresh or check IBM Quantum dashboard.")
        return

    # Verify bitstring length
    print(f"Raw bitstring length: {len(all_bits)} bits")
    if len(all_bits) < total_bits_needed:
        print(f"Warning: Only got {len(all_bits)} bits, needed {total_bits_needed}. Padding with zeros.")

    # Process bits into integers and doubles
    bitstring = all_bits + "0" * (total_bits_needed - len(all_bits))  # Pad if needed
    integers = bits_to_integers(bitstring)
    doubles = integers_to_doubles(integers)

    # Ensure exactly 500 numbers
    print(f"Generated {len(integers)} integers")
    if len(integers) != num_samples:
        print(f"Error: Expected {num_samples} integers, got {len(integers)}. Results may be incomplete.")

    # Prepare data for CSV
    data = []
    for i in range(num_samples):
        data.append({
            'index': i,
            'integer': integers[i] if i < len(integers) else 0,
            'double': doubles[i] if i < len(doubles) else 0.0,
            'source': 'Quantum_Hardware',
            'timestamp': datetime.now().timestamp() + (i * 1e-6)  # Increment timestamp slightly
        })

    # Save to CSV
    output_file = 'quantum_hardware_data.csv'
    try:
        with open(output_file, 'w', newline='') as csvfile:
            fieldnames = ['index', 'integer', 'double', 'source', 'timestamp']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)
        print(f"Hardware Quantum data saved to {output_file}")
        print("To download, check the Files tab on the left in Colab.")
    except IOError as e:
        print(f"Error writing to file: {e}")
        return

    # Calculate and display metrics
    execution_time = time.time() - start_time
    throughput = total_bits_needed / execution_time if execution_time > 0 else 0
    print(f"Execution time (submission to result): {execution_time:.2f} seconds")
    print(f"Throughput: {throughput:.2f} bits/second")
    print("Run complete! Compare with simulator and PRNG data for analysis.")

if __name__ == "__main__":
    try:
        run_quantum_hardware_qrng()
    except Exception as e:
        print(f"An unanticipated error occurred: {e}")

In [ ]:
# Install required packages for analysis
!pip install numpy==1.26.4 pandas==2.2.2 scipy==1.13.1 statsmodels==0.14.2 matplotlib==3.7.1 seaborn==0.13.2 scikit-learn==1.5.1

import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import os
import warnings

# Suppress RuntimeWarnings
warnings.filterwarnings('ignore', message='invalid value encountered in multiply')

# Load CSV files for PRNG, Simulated Quantum, and Hardware Quantum
files = ['prng_data.csv', 'quantum_data.csv', 'quantum_hardware_data.csv']
dataframes = []
for f in files:
    if os.path.exists(f):
        df = pd.read_csv(f)
        # Truncate or pad to 500 samples for consistency (matching hardware output)
        if len(df) > 500:
            df = df.iloc[:500]
        elif len(df) < 500:
            additional = 500 - len(df)
            df = pd.concat([df, df.sample(n=additional, replace=True, random_state=42)], ignore_index=True)
        dataframes.append(df)
        print(f"Loaded {f} with {len(df)} samples (adjusted to 500)")
    else:
        print(f"Warning: {f} not found, skipping.")
if not dataframes:
    print("Error: No data files found.")
    exit(1)
data = pd.concat(dataframes, ignore_index=True)

# Read execution times with fallbacks
exec_times = {
    'PRNG': None,
    'Quantum': None,
    'Quantum_Hardware': None
}
for method, fname in [('PRNG', 'prng_time.txt'), ('Quantum', 'quantum_time.txt'), ('Quantum_Hardware', 'quantum_hardware_time.txt')]:
    if os.path.exists(fname):
        with open(fname, 'r') as f:
            exec_times[method] = float(f.read().strip())
    else:
        exec_times[method] = {'PRNG': 0.27, 'Quantum': 7.8, 'Quantum_Hardware': 280.34}.get(method, None)
        print(f"Warning: {fname} not found, using fallback: {exec_times[method]} seconds")

# Bootstrapping function
def bootstrap_metric(data, metric_func, n_bootstraps=1000):
    """Perform bootstrapping on data to compute mean and 95% confidence intervals."""
    boot_results = []
    for _ in range(n_bootstraps):
        sample = np.random.choice(data, size=len(data), replace=True)
        try:
            result = metric_func(sample)
            if not np.isnan(result):
                boot_results.append(result)
        except Exception as e:
            print(f"Warning: Bootstrap failure for {metric_func.__name__}: {e}")
            continue
    if not boot_results:
        return np.nan, (np.nan, np.nan)
    mean = np.mean(boot_results)
    ci = stats.t.interval(0.95, df=len(boot_results)-1, loc=mean, scale=stats.sem(boot_results)) if len(boot_results) > 1 else (np.nan, np.nan)
    return mean, ci

# Quality score function
def quality_score(entropy, chi2_p, autocorr):
    """Compute a quality score based on raw randomness metrics."""
    return 0.4 * entropy + 0.4 * chi2_p - 0.2 * np.abs(autocorr)

# Analysis functions
def calculate_throughput(ints, exec_time):
    """Calculate throughput in bits per second."""
    return len(ints) * 32 / exec_time if exec_time else None

def analyze_method(source, ints, doubles, exec_time):
    # Entropy for integers
    int_bytes = ints.astype(np.uint32).tobytes()
    p = np.bincount(np.frombuffer(int_bytes, dtype=np.uint8), minlength=256) / len(int_bytes)
    entropy_int = -np.sum(p[p > 0] * np.log2(p[p > 0]))

    # Chi-square test (20 bins for better resolution)
    bin_edges = np.linspace(0, 2**32, 21)
    observed, _ = np.histogram(ints, bins=bin_edges)
    expected = np.full(20, len(ints) / 20)
    chi2, p_value = stats.chisquare(observed, expected)

    # Autocorrelation (lag-1)
    autocorr = acf(doubles, nlags=1, fft=True)[1]

    # Throughput
    throughput = calculate_throughput(ints, exec_time)

    # Bootstrapped metrics
    entropy_boot, entropy_ci = bootstrap_metric(
        ints,
        lambda x: -np.sum(
            (p := np.bincount(np.frombuffer(x.astype(np.uint32).tobytes(), dtype=np.uint8), minlength=256) / len(x.astype(np.uint32).tobytes())) *
            np.log2(np.where(p > 0, p, 1))
        )
    )
    chi2_p_boot, chi2_p_ci = bootstrap_metric(
        ints,
        lambda x: stats.chisquare(np.histogram(x, bins=bin_edges)[0], expected)[1]
    )
    autocorr_boot, autocorr_ci = bootstrap_metric(
        doubles,
        lambda x: acf(x, nlags=1, fft=True)[1]
    )

    # Quality scores
    quality = quality_score(entropy_int, p_value, autocorr)
    quality_boot = quality_score(entropy_boot, chi2_p_boot, autocorr_boot)

    return [source, entropy_int, p_value, autocorr, throughput, exec_time, quality,
            entropy_boot, entropy_ci, chi2_p_boot, chi2_p_ci, autocorr_boot, autocorr_ci, quality_boot]

# Perform analysis
results = []
for source in ['PRNG', 'Quantum', 'Quantum_Hardware']:
    subset = data[data['source'] == source]
    ints = subset['integer'].values
    doubles = subset['double'].values
    exec_time = exec_times.get(source)
    result = analyze_method(source, ints, doubles, exec_time)
    results.append(result)

# Normalize quality scores across all methods
scaler = MinMaxScaler()
quality_values = [r[6] for r in results] + [r[13] for r in results]
normalized_qualities = scaler.fit_transform(np.array(quality_values).reshape(-1, 1)).flatten()
for i, r in enumerate(results):
    r[6] = normalized_qualities[i]
    r[13] = normalized_qualities[i + len(results)]

# Save results
columns = ['method', 'entropy_int', 'chi_square_p', 'autocorr_lag1', 'throughput', 'exec_time', 'quality',
           'entropy_boot', 'entropy_ci', 'chi2_p_boot', 'chi2_p_ci', 'autocorr_boot', 'autocorr_ci', 'quality_boot']
pd.DataFrame(results, columns=columns).to_csv('prng_quantum_analysis_summary.csv', index=False)

# Visualize results
# Histogram of Integers
plt.figure(figsize=(12, 6))
for i, source in enumerate(['PRNG', 'Quantum', 'Quantum_Hardware']):
    plt.hist(data[data['source'] == source]['integer'], bins=100, alpha=0.5, density=False, label=source, color=plt.cm.Set1(i))
plt.legend()
plt.title('Histogram of Integers: PRNG vs. Quantum (Simulated & Hardware)')
plt.xlabel('Integer Value (0–4,294,967,295)')
plt.ylabel('Frequency')
plt.yscale('log')
plt.savefig('prng_quantum_integers_hist_enhanced.png')
plt.close()

# Autocorrelation Plot
plt.figure(figsize=(12, 6))
for i, source in enumerate(['PRNG', 'Quantum', 'Quantum_Hardware']):
    doubles = data[data['source'] == source]['double'].values
    autocorr = acf(doubles, nlags=20, fft=True)
    n = len(doubles)
    ci = 1.96 / np.sqrt(n)
    plt.plot(range(21), autocorr, label=source, color=plt.cm.Set1(i))
    plt.fill_between(range(21), autocorr - ci, autocorr + ci, alpha=0.2, color=plt.cm.Set1(i))
plt.legend()
plt.title('Autocorrelation of Doubles: PRNG vs. Quantum (Lags 0–20)')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.savefig('prng_quantum_autocorr_plot_enhanced.png')
plt.close()

# Statistical tests
print("\nKolmogorov-Smirnov Test for Uniformity (PRNG vs. Quantum):")
for source in ['PRNG', 'Quantum', 'Quantum_Hardware']:
    subset = data[data['source'] == source]['integer'].values / 2**32
    stat, p = stats.kstest(subset, 'uniform')
    print(f"{source}: Statistic = {stat:.4f}, p-value = {p:.4f}")

print("\nAnalysis Summary (PRNG vs. Quantum):")
for r in results:
    print(f"Method: {r[0]}")
    print(f"  Entropy (int): {r[1]:.4f} bits/byte")
    print(f"  Chi-square p-value: {r[2]:.4f}")
    print(f"  Autocorr lag-1: {r[3]:.4f}")
    print(f"  Throughput: {r[4]:.2f} bits/second" if r[4] else "  Throughput: N/A")
    print(f"  Exec time: {r[5]:.2f} seconds" if r[5] else "  Exec time: N/A")
    print(f"  Quality score: {r[6]:.4f}")
    print(f"  Bootstrapped Entropy (int): {r[7]:.4f} ({r[8][0]:.4f} - {r[8][1]:.4f})")
    print(f"  Bootstrapped Chi-square p-value: {r[9]:.4f} ({r[10][0]:.4f} - {r[10][1]:.4f})")
    print(f"  Bootstrapped Autocorr lag-1: {r[11]:.4f} ({r[12][0]:.4f} - {r[12][1]:.4f})")
    print(f"  Bootstrapped Quality score: {r[13]:.4f}")